In [5]:
!python --version
!pip install python-dotenv

Python 3.8.5


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='Users/gregorykatz/GPT-Azure-Search-Engine/credentials.env')

print(os.getenv('AZURE_SEARCH_KEY'))


In [62]:
!pip install openpyxl
!pip install --upgrade azure-storage-blob
!pip install openai==0.28.1
!pip install --upgrade typing_extensions


Requirement already up-to-date: azure-storage-blob in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (12.19.0)
Requirement already up-to-date: typing_extensions in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (4.9.0)


In [ ]:
BLOB_CONNECTION_STRING=os.getenv('BLOB_CONNECTION_STRING')
BLOB_CONTAINER_NAME=os.getenv('BLOB_CONTAINER_NAME')
AZURE_SEARCH_ENDPOINT=os.getenv('AZURE_SEARCH_ENDPOINT')
AZURE_SEARCH_KEY=os.getenv('AZURE_SEARCH_KEY')
AZURE_OPENAI_API_KEY4=os.getenv('AZURE_OPENAI_API_KEY4')
AZURE_OPENAI_ENDPOINT4=os.getenv('AZURE_OPENAI_ENDPOINT4')
processed_container=os.getenv('processed_container')
raw_container_processed=os.getenv('raw_container_processed')
print(BLOB_CONNECTION_STRING)
print(processed_container)

In [63]:
# Import necessary libraries
from azure.storage.blob import BlobServiceClient, BlobClient
import pandas as pd
import os


# Define your container names
processed_container = 'surveyprocessed'
raw_container_processed = 'surveyrawprocessed'

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(os.getenv('BLOB_CONNECTION_STRING'))

# Access the source container
source_container_client = blob_service_client.get_container_client(os.getenv('BLOB_CONTAINER_NAME'))

# Loop through each blob in the container
for blob in source_container_client.list_blobs():
    if blob.name.endswith('.xlsx'):
        # Download the blob to a local file
        downloader = source_container_client.download_blob(blob.name)
        with open(blob.name, 'wb') as file:
            file.write(downloader.readall())
            

        # Load the Excel file into a DataFrame
        df = pd.read_excel(blob.name, engine='openpyxl')

        # Get all the columns that start with 'Mission'
        mission_cols = [col for col in df.columns if col.startswith('Mission')]

        # Define a function to return the column name if its value is 'x'
        def get_mission(row):
            missions = [col.split('||')[1].strip() for col in mission_cols if row[col] == 'x']
            return missions if missions else None

        # Apply the function to each row
        df['mission_defined'] = df.apply(get_mission, axis=1)

        # Get all the columns that start with 'Natural Theme ||'
        theme_cols = [col for col in df.columns if col.startswith('Natural Theme ||')]

        # Define a function to return the column name if its value is 'x'
        def get_theme(row):
            themes = [col.split('||')[1].strip() for col in theme_cols if row[col] == 'x']
            return themes if themes else None

        # Apply the function to each row
        df['theme_defined'] = df.apply(get_theme, axis=1)

        # Replace 'x' with 'Yes' and NaN values with 'No' in the 'Large-Scale Initiative || Digital Technology' column
        df['Large-Scale Initiative || Digital Technology'] = df['Large-Scale Initiative || Digital Technology'].replace('x', 'Yes').fillna('No')

        # Drop the 'Mission' and 'Natural Theme ||' columns from the DataFrame
        drop_cols = [col for col in df.columns if col.startswith('Mission') or col.startswith('Natural Theme ||')]
        df = df.drop(columns=drop_cols)

        # Now df is your DataFrame this is out of loop will need to change code to process more in loop so multiple file processed. 

        from IPython.display import display, HTML

        # Convert the DataFrame to an HTML table with borders
        html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

        # Display the HTML table
        display(HTML(html_table))




,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,mission_defined,theme_defined
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The waiting times in the emergency department are too long. This often leads to patient discomfort and dissatisfaction.,yes,solution,Yes,None,[[Theme 1]]
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A triage system could help prioritize cases in the emergency department. It would ensure that patients with the most urgent needs are attended to first.,no,neither,No,None,None
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The hospital is understaffed during night shifts. This can lead to delays in patient care and increased workload for existing staff.,no,NaN,Yes,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]"
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring more night shift nurses could alleviate staffing issues. It would ensure adequate care for all patients and reduce burnout among existing staff.,yes,NaN,Yes,[Research & Discovery],[[Theme 1]]
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The hospital food lacks variety and taste. Patients often complain about the lack of healthy and appetizing options.,no,NaN,No,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]"
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The hospital’s parking lot is often full, making it difficult for visitors. This can cause stress for visitors and may deter them from visiting their loved ones.",yes,NaN,Yes,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]"
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A menu revamp could improve patient satisfaction. Offering a variety of tasty and nutritious meals could enhance the overall patient experience.,yes,NaN,No,[Teaching],[[Theme 6]]
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A multi-level parking garage could solve the parking issue. It would provide ample parking space for visitors and staff.,yes,NaN,No,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]"
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The hospital rooms are not cleaned regularly. This can lead to unsanitary conditions and potential health risks for patients.,yes,NaN,No,None,None
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing the frequency of cleaning could improve cleanliness. Regular cleaning and sanitation are crucial for maintaining a healthy hospital environment.,yes,NaN,No,None,None


In [64]:
# Import necessary libraries
from azure.storage.blob import BlobServiceClient, BlobClient
import pandas as pd
import os

# Set up your environment variables
os.environ['BLOB_CONNECTION_STRING'] = "DefaultEndpointsProtocol=https;AccountName=adlsgenstor2;AccountKey=7/nueSoBn3wheMkA419GImZ8KLiU1B7Flfnr9g1JP0+JRCe/2eenJig9avdodiRDM73UwDqwcrUV+AStCSIbDw==;EndpointSuffix=core.windows.net"
os.environ['BLOB_CONTAINER_NAME'] = "survey"
os.environ['AZURE_SEARCH_ENDPOINT'] = "https://cog-search-gu6guw3t3tdvs.search.windows.net"
os.environ['AZURE_SEARCH_KEY'] = "bdfGLxc15S2PnIOQSU5Kd9lY8mwmXTg6RMF0IoMfIvAzSeAurfbf"
os.environ['AZURE_OPENAI_ENDPOINT4'] = "https://aiinstancecan.openai.azure.com/"
os.environ['AZURE_OPENAI_API_KEY4'] = "88ca7bb6a3474909b38063ba2b3af5a4"

# Define your container names
processed_container = 'surveyprocessed'
raw_container_processed = 'surveyrawprocessed'

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(os.getenv('BLOB_CONNECTION_STRING'))

# Access the source container
source_container_client = blob_service_client.get_container_client(os.getenv('BLOB_CONTAINER_NAME'))

# Loop through each blob in the container
for blob in source_container_client.list_blobs():
    if blob.name.endswith('.xlsx'):
        # Download the blob to a local file
        downloader = source_container_client.download_blob(blob.name)
        with open(blob.name, 'wb') as file:
            file.write(downloader.readall())
            

        # Load the Excel file into a DataFrame
        df = pd.read_excel(blob.name, engine='openpyxl')

        # Add a new column to your DataFrame
        df['gpt_problem'] = ''

        # Get all the columns that start with 'Mission'
        mission_cols = [col for col in df.columns if col.startswith('Mission')]

        # Define a function to return the column name if its value is 'x'
        def get_mission(row):
            missions = [col.split('||')[1].strip() for col in mission_cols if row[col] == 'x']
            return missions if missions else None

        # Apply the function to each row
        df['mission_defined'] = df.apply(get_mission, axis=1)

        # Get all the columns that start with 'Natural Theme ||'
        theme_cols = [col for col in df.columns if col.startswith('Natural Theme ||')]

        # Define a function to return the column name if its value is 'x'
        def get_theme(row):
            themes = [col.split('||')[1].strip() for col in theme_cols if row[col] == 'x']
            return themes if themes else None

        # Apply the function to each row
        df['theme_defined'] = df.apply(get_theme, axis=1)

        # Replace 'x' with 'Yes' and NaN values with 'No' in the 'Large-Scale Initiative || Digital Technology' column
        df['Large-Scale Initiative || Digital Technology'] = df['Large-Scale Initiative || Digital Technology'].replace('x', 'Yes').fillna('No')

        # Drop the 'Mission' and 'Natural Theme ||' columns from the DataFrame
        drop_cols = [col for col in df.columns if col.startswith('Mission') or col.startswith('Natural Theme ||')]
        df = df.drop(columns=drop_cols)

        # Now df is your DataFrame this is out of loop will need to change code to process more in loop so multiple file processed. 

        from IPython.display import display, HTML

        # Convert the DataFrame to an HTML table with borders
        html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

        # Display the HTML table
        display(HTML(html_table))


,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The waiting times in the emergency department are too long. This often leads to patient discomfort and dissatisfaction.,yes,solution,Yes,,None,[[Theme 1]]
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A triage system could help prioritize cases in the emergency department. It would ensure that patients with the most urgent needs are attended to first.,no,neither,No,,None,None
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The hospital is understaffed during night shifts. This can lead to delays in patient care and increased workload for existing staff.,no,NaN,Yes,,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]"
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring more night shift nurses could alleviate staffing issues. It would ensure adequate care for all patients and reduce burnout among existing staff.,yes,NaN,Yes,,[Research & Discovery],[[Theme 1]]
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The hospital food lacks variety and taste. Patients often complain about the lack of healthy and appetizing options.,no,NaN,No,,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]"
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The hospital’s parking lot is often full, making it difficult for visitors. This can cause stress for visitors and may deter them from visiting their loved ones.",yes,NaN,Yes,,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]"
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A menu revamp could improve patient satisfaction. Offering a variety of tasty and nutritious meals could enhance the overall patient experience.,yes,NaN,No,,[Teaching],[[Theme 6]]
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A multi-level parking garage could solve the parking issue. It would provide ample parking space for visitors and staff.,yes,NaN,No,,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]"
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The hospital rooms are not cleaned regularly. This can lead to unsanitary conditions and potential health risks for patients.,yes,NaN,No,,None,None
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing the frequency of cleaning could improve cleanliness. Regular cleaning and sanitation are crucial for maintaining a healthy hospital environment.,yes,NaN,No,,None,None


In [65]:
!pip install spaCy
!python -m spacy validate
!python -m spacy download en_core_web_sm


✔ Loaded compatibility table

====================== Installed models (spaCy v2.2.4) ======================
ℹ spaCy installation:
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.2.5   ✔

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [19]:
import os
import openai
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = AZURE_OPENAI_ENDPOINT4 # Your Azure OpenAI resource's endpoint value.
openai.api_key = AZURE_OPENAI_API_KEY4

response = openai.ChatCompletion.create(
    engine="gpt-35-turbo", # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)

print(response)

# To print only the response content text:
# print(response['choices'][0]['message']['content'])

{
  "id": "chatcmpl-8nolwfcYGGNk1rktJ7m0uZUX6M4e4",
  "object": "chat.completion",
  "created": 1706883932,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Microsoft was founded by Bill Gates and Paul Allen."
      }
    }
  ],
  "usage": {
    "prompt_tokens": 29,
    "completion_tokens": 10,
    "total_tokens": 39
  }
}


In [66]:
import os
import openai
import numpy as np
import pandas as pd
import time

openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = AZURE_OPENAI_ENDPOINT4 # Your Azure OpenAI resource's endpoint value.
openai.api_key = AZURE_OPENAI_API_KEY4

import concurrent.futures

# Define a function to process a single comment
def process_comment(comment):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that categorizes comments into 'problem', 'solution', 'neither', or 'both'. A problem could be a negative statement like 'waiting times are long' or 'the hospital rooms aren't cleaned regularly'. A solution could be a suggestion for improvement like 'reduce waiting times' or 'clean the hospital rooms more frequently'. If a comment implies a problem through the solution, it should be categorized as 'both'. If a comment is just a complaint or doesn't fit into these categories, it should be categorized as 'neither'. Please remember to identify comments that imply both a problem and a solution as 'both'."
        },
        {
            "role": "user",
            "content": comment
        }
    ]
    # Make a chat completion Request for the API
    response = openai.ChatCompletion.create(
        engine="gpt-4-32k", # The deployment name you chose when you deployed the GPT-4 model.
        messages=messages
    )
    # Process the response from API
    reply = response['choices'][0]['message']['content'].strip()
    # Return the reply
    return reply

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Break the DataFrame into smaller batches
    batch_size = 25 # Adjust this value based on your needs
    df_batches = np.array_split(df, len(df) // batch_size)
    
    df_list = [] # List to store the processed DataFrame batches
    
    for i, df_batch in enumerate(df_batches):
        # Use the executor to map the process_comment function to the comments in the DataFrame batch
        replies = list(executor.map(process_comment, df_batch['Participant Comment']))
        # Add the replies to the DataFrame batch
        df_batch['gpt_problem'] = replies
        df_list.append(df_batch) # Add the processed DataFrame batch to the list

        print(f"Processed {batch_size} lines of survey") # Print a message after each batch is processed
        time.sleep(5) # Wait for 5 seconds before processing the next batch

# Concatenate all the processed DataFrame batches back into a single DataFrame
df = pd.concat(df_list)


Processed 25 lines of survey
Processed 25 lines of survey
Processed 25 lines of survey
Processed 25 lines of survey
Processed 25 lines of survey
Processed 25 lines of survey
Processed 25 lines of survey


In [67]:
        from IPython.display import display, HTML

        # Convert the DataFrame to an HTML table with borders
        html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

        # Display the HTML table
        display(HTML(html_table))

,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The waiting times in the emergency department are too long. This often leads to patient discomfort and dissatisfaction.,yes,solution,Yes,Problem,None,[[Theme 1]]
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A triage system could help prioritize cases in the emergency department. It would ensure that patients with the most urgent needs are attended to first.,no,neither,No,Solution,None,None
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The hospital is understaffed during night shifts. This can lead to delays in patient care and increased workload for existing staff.,no,NaN,Yes,Problem,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]"
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring more night shift nurses could alleviate staffing issues. It would ensure adequate care for all patients and reduce burnout among existing staff.,yes,NaN,Yes,both,[Research & Discovery],[[Theme 1]]
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The hospital food lacks variety and taste. Patients often complain about the lack of healthy and appetizing options.,no,NaN,No,Problem,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]"
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The hospital’s parking lot is often full, making it difficult for visitors. This can cause stress for visitors and may deter them from visiting their loved ones.",yes,NaN,Yes,Problem,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]"
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A menu revamp could improve patient satisfaction. Offering a variety of tasty and nutritious meals could enhance the overall patient experience.,yes,NaN,No,Both,[Teaching],[[Theme 6]]
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A multi-level parking garage could solve the parking issue. It would provide ample parking space for visitors and staff.,yes,NaN,No,both,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]"
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The hospital rooms are not cleaned regularly. This can lead to unsanitary conditions and potential health risks for patients.,yes,NaN,No,problem,None,None
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing the frequency of cleaning could improve cleanliness. Regular cleaning and sanitation are crucial for maintaining a healthy hospital environment.,yes,NaN,No,both,None,None


In [69]:
def categorize_comment(comment):
    comment = comment.lower()
    if 'problem' in comment and 'solution' in comment:
        return 'both'
    elif 'problem' in comment:
        return 'problem'
    elif 'solution' in comment:
        return 'solution'
    else:
        return 'neither'

# Apply the function to the 'gpt_problem' column
df['gpt_problem'] = df['gpt_problem'].apply(categorize_comment)


In [70]:
        from IPython.display import display, HTML

        # Convert the DataFrame to an HTML table with borders
        html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

        # Display the HTML table
        display(HTML(html_table))

,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The waiting times in the emergency department are too long. This often leads to patient discomfort and dissatisfaction.,yes,solution,Yes,problem,None,[[Theme 1]]
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A triage system could help prioritize cases in the emergency department. It would ensure that patients with the most urgent needs are attended to first.,no,neither,No,solution,None,None
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The hospital is understaffed during night shifts. This can lead to delays in patient care and increased workload for existing staff.,no,NaN,Yes,problem,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]"
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring more night shift nurses could alleviate staffing issues. It would ensure adequate care for all patients and reduce burnout among existing staff.,yes,NaN,Yes,neither,[Research & Discovery],[[Theme 1]]
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The hospital food lacks variety and taste. Patients often complain about the lack of healthy and appetizing options.,no,NaN,No,problem,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]"
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The hospital’s parking lot is often full, making it difficult for visitors. This can cause stress for visitors and may deter them from visiting their loved ones.",yes,NaN,Yes,problem,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]"
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A menu revamp could improve patient satisfaction. Offering a variety of tasty and nutritious meals could enhance the overall patient experience.,yes,NaN,No,neither,[Teaching],[[Theme 6]]
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A multi-level parking garage could solve the parking issue. It would provide ample parking space for visitors and staff.,yes,NaN,No,neither,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]"
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The hospital rooms are not cleaned regularly. This can lead to unsanitary conditions and potential health risks for patients.,yes,NaN,No,problem,None,None
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing the frequency of cleaning could improve cleanliness. Regular cleaning and sanitation are crucial for maintaining a healthy hospital environment.,yes,NaN,No,neither,None,None


In [71]:
import os
import openai
import numpy as np
import pandas as pd
import time

openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = AZURE_OPENAI_ENDPOINT4 # Your Azure OpenAI resource's endpoint value.
openai.api_key = AZURE_OPENAI_API_KEY4

import concurrent.futures

# Define a function to process a single comment
def process_comment(comment):
    messages = [
        {
            "role": "system",
            "content": "Which mission category does this comment belong to: Patient Care, Research & Discovery, Teaching, Community, Quality & Safety, Leadership, Equity, Partnership, Innovation, Advocacy, or Other?"
        },
        {
            "role": "user",
            "content": comment
        }
    ]
    # Make a chat completion Request for the API
    response = openai.ChatCompletion.create(
        engine="gpt-4-32k", # The deployment name you chose when you deployed the GPT-4 model.
        messages=messages
    )
    # Process the response from API
    reply = response['choices'][0]['message']['content'].strip().lower()
    # Return the reply
    return reply

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Break the DataFrame into smaller batches
    batch_size = 10 # Adjust this value based on your needs
    df_batches = np.array_split(df, len(df) // batch_size)
    
    df_list = [] # List to store the processed DataFrame batches
    
    for i, df_batch in enumerate(df_batches):
        # Use the executor to map the process_comment function to the comments in the DataFrame batch
        replies = list(executor.map(process_comment, df_batch['Participant Comment']))
        # Add the replies to the DataFrame batch
        df_batch['gpt_Mission'] = replies
        df_list.append(df_batch) # Add the processed DataFrame batch to the list

        print(f"Processed {batch_size} lines of survey") # Print a message after each batch is processed
        time.sleep(10) # Wait for 5 seconds before processing the next batch

# Concatenate all the processed DataFrame batches back into a single DataFrame
df = pd.concat(df_list)


Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey


In [84]:
        from IPython.display import display, HTML

        # Convert the DataFrame to an HTML table with borders
        html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

        # Display the HTML table
        display(HTML(html_table))

,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined,gpt_Mission,Natural_Theme,Natural_Subtheme
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The Waiting Times In The Emergency Department Are Too Long. This Often Leads To Patient Discomfort And Dissatisfaction.,Yes,Solution,Yes,Problem,None,[[Theme 1]],Quality & Safety,None,None
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A Triage System Could Help Prioritize Cases In The Emergency Department. It Would Ensure That Patients With The Most Urgent Needs Are Attended To First.,No,Neither,No,Solution,None,None,Quality & Safety,None,None
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The Hospital Is Understaffed During Night Shifts. This Can Lead To Delays In Patient Care And Increased Workload For Existing Staff.,No,NaN,Yes,Problem,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]",Quality & Safety,None,None
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring More Night Shift Nurses Could Alleviate Staffing Issues. It Would Ensure Adequate Care For All Patients And Reduce Burnout Among Existing Staff.,Yes,NaN,Yes,Neither,[Research & Discovery],[[Theme 1]],"Patient Care, Quality & Safety, Leadership",None,None
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The Hospital Food Lacks Variety And Taste. Patients Often Complain About The Lack Of Healthy And Appetizing Options.,No,NaN,No,Problem,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]",Patient Care,None,None
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The Hospital’S Parking Lot Is Often Full, Making It Difficult For Visitors. This Can Cause Stress For Visitors And May Deter Them From Visiting Their Loved Ones.",Yes,NaN,Yes,Problem,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]",Quality & Safety,None,None
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A Menu Revamp Could Improve Patient Satisfaction. Offering A Variety Of Tasty And Nutritious Meals Could Enhance The Overall Patient Experience.,Yes,NaN,No,Neither,[Teaching],[[Theme 6]],Patient Care,None,None
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A Multi-Level Parking Garage Could Solve The Parking Issue. It Would Provide Ample Parking Space For Visitors And Staff.,Yes,NaN,No,Neither,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]",Other,None,None
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The Hospital Rooms Are Not Cleaned Regularly. This Can Lead To Unsanitary Conditions And Potential Health Risks For Patients.,Yes,NaN,No,Problem,None,None,Quality & Safety,None,None
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing The Frequency Of Cleaning Could Improve Cleanliness. Regular Cleaning And Sanitation Are Crucial For Maintaining A Healthy Hospital Environment.,Yes,NaN,No,Neither,None,None,Quality & Safety,None,None


In [77]:
import warnings
import numpy as np
warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


In [96]:
# cell to be removed
def process_comment(comment):
    themes = [
        "Ambulatory: Operations;Referral Management;Quality",
        "Buildings and Grounds: Maintenance;Cleaning;Security;Accessibility",
        "Communications: Documentation;Email;Paging;Patient Gateway",
        "Cost Management: Cost Savings:Cost Avoidance",
        "Credentialing: Verification;Privileging;Re-credentialing;Provider Enrollment",
        "Documentation: Medical Records;Coding;Compliance;Privacy",
        "Environment Services Depart.: Cleaning",
        "Epic: Communications;billing;break the glass;charting;documentation;dragon;imaging;inbasket;login, etc.",
        "Food and Natural Services",
        "Human Resources: Hiring;Training;Benefits;Employee Relations",
        "Infectious Disease: Prevention;Control;Outbreak Management;Vaccination",
        "IT: Hardware;Software;Network;Security",
        "Pharmacy: Medication Management;Dispensing;Consultation;Compliance",
        "Patient Experience: Satisfaction;Comfort;Privacy;Communication with Staff;Wait Times",
        "Medical Equipment: Availability;Maintenance;Upgrades;Training",
        "Emergency Services: Response Time;Staffing;Equipment;Training",
        "Surgical Services: Preoperative Care;Postoperative Care;Staffing;Equipment",
        "Diagnostic Services: Imaging;Laboratory;Pathology;Radiology",
        "Therapeutic Services: Physical Therapy;Occupational Therapy;Speech Therapy;Respiratory Therapy",
        "Support Services: Transport;Security;Interpretation Services;Social Work",
        "Administrative Services: Billing;Insurance;Registration;Scheduling",
        "Outpatient Services: Scheduling;Wait Times;Staffing;Facilities",
        "Inpatient Services: Room Quality;Nursing Care;Meals;Visitation Policies",
        "Mental Health Services: Counseling;Psychiatry;Support Groups;Crisis Intervention",
        "Patient Rights: Privacy;Informed Consent;Access to Medical Records"
    ]

    theme_list = []
    subtheme_list = []

    for theme_entry in themes:
        parts = theme_entry.split(":")
        if len(parts) >= 2:
            theme = parts[0].strip()
            subthemes = [subtheme.strip() for subtheme in parts[1].split(";")]

            # Check if the theme or any subthemes appear in the comment
            if theme in comment or any(subtheme in comment for subtheme in subthemes):
                theme_list.append(theme)
                subtheme_list.append(subthemes)

    return theme_list, subtheme_list

 

In [98]:
# cell to be removed
theme_list, subtheme_list = process_comment("I had a problem with the Ambulatory Operations")
print(theme_list)
print(subtheme_list)


['Ambulatory']
[['Operations', 'Referral Management', 'Quality']]


In [99]:
import os
import openai
import numpy as np
import pandas as pd
import time
import concurrent.futures

openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = AZURE_OPENAI_ENDPOINT4 # Your Azure OpenAI resource's endpoint value.
openai.api_key = AZURE_OPENAI_API_KEY4

# Define a function to process a single comment
def process_comment(comment):
    themes = [
        "Ambulatory: Operations;Referral Management;Quality",
        "Buildings and Grounds: Maintenance;Cleaning;Security;Accessibility",
        "Communications: Documentation;Email;Paging;Patient Gateway",
        "Cost Management: Cost Savings:Cost Avoidance",
        "Credentialing: Verification;Privileging;Re-credentialing;Provider Enrollment",
        "Documentation: Medical Records;Coding;Compliance;Privacy",
        "Environment Services Depart.: Cleaning",
        "Epic: Communications;billing;break the glass;charting;documentation;dragon;imaging;inbasket;login, etc.",
        "Food and Natural Services",
        "Human Resources: Hiring;Training;Benefits;Employee Relations",
        "Infectious Disease: Prevention;Control;Outbreak Management;Vaccination",
        "IT: Hardware;Software;Network;Security",
        "Pharmacy: Medication Management;Dispensing;Consultation;Compliance",
        "Patient Experience: Satisfaction;Comfort;Privacy;Communication with Staff;Wait Times",
        "Medical Equipment: Availability;Maintenance;Upgrades;Training",
        "Emergency Services: Response Time;Staffing;Equipment;Training",
        "Surgical Services: Preoperative Care;Postoperative Care;Staffing;Equipment",
        "Diagnostic Services: Imaging;Laboratory;Pathology;Radiology",
        "Therapeutic Services: Physical Therapy;Occupational Therapy;Speech Therapy;Respiratory Therapy",
        "Support Services: Transport;Security;Interpretation Services;Social Work",
        "Administrative Services: Billing;Insurance;Registration;Scheduling",
        "Outpatient Services: Scheduling;Wait Times;Staffing;Facilities",
        "Inpatient Services: Room Quality;Nursing Care;Meals;Visitation Policies",
        "Mental Health Services: Counseling;Psychiatry;Support Groups;Crisis Intervention",
        "Patient Rights: Privacy;Informed Consent;Access to Medical Records;Complaint Procedures",
        "Healthcare Accessibility: Language Services;Disability Accommodations;Transportation Services;Telemedicine",
        "Preventive Care: Screenings;Immunizations;Health Education;Wellness Programs",
        "Specialty Care: Cardiology;Oncology;Neurology;Orthopedics;Pediatrics",
        "Palliative Care: Pain Management;End-of-Life Care;Support for Families",
        "Rehabilitation Services: Physical Rehabilitation;Occupational Rehabilitation;Speech and Language Therapy",
        "Maternal and Child Health: Prenatal Care;Pediatric Care;Neonatal Intensive Care;Breastfeeding Support",
        "Health Promotion: Disease Prevention;Nutrition Counseling;Exercise Programs;Smoking Cessation",
        "Patient Engagement: Patient Portals;Shared Decision Making;Patient Satisfaction Surveys;Patient Advisory Councils"
    ]

    theme_list = []
    subtheme_list = []

    for theme_entry in themes:
        parts = theme_entry.split(":")
        if len(parts) >= 2:
            theme = parts[0].strip()
            subthemes = [subtheme.strip() for subtheme in parts[1].split(";")]

            # Check if the theme or any subthemes appear in the comment
            if theme in comment or any(subtheme in comment for subtheme in subthemes):
                theme_list.append(theme)
                subtheme_list.append(subthemes)

    messages = [
        {
            "role": "system",
            "content": f"Which natural theme and subcategory does this comment belong to: {', '.join(themes)}?"
        },
        {
            "role": "user",
            "content": comment
        }
    ]
    # Make a chat completion Request for the API
    response = openai.ChatCompletion.create(
        engine="gpt-4-32k", # The deployment name you chose when you deployed the GPT-4 model.
        messages=messages
    )
    # Process the response from API
    gpt_generated_reply = response['choices'][0]['message']['content'].strip().lower()
    # Extract only the categories from the reply
    gpt_generated_theme, gpt_generated_subtheme = gpt_generated_reply.split(':', 1) if ':' in gpt_generated_reply else (gpt_generated_reply, None)

    # Check if the GPT-generated theme and subtheme are in the predefined list
    if f"{gpt_generated_theme}:{gpt_generated_subtheme}" not in themes:
        gpt_generated_theme, gpt_generated_subtheme = None, None

    # Combine the predefined and GPT-generated themes and subthemes
    theme = theme_list[0] if theme_list else gpt_generated_theme
    subtheme = subtheme_list[0] if subtheme_list else gpt_generated_subtheme

    # Return the combined theme and subtheme
    return theme, subtheme

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Break the DataFrame into smaller batches
    batch_size = 10 # Adjust this value based on your needs
    df_batches = np.array_split(df, len(df) // batch_size)
    
    df_list = [] # List to store the processed DataFrame batches
    
    for i, df_batch in enumerate(df_batches):
        # Use the executor to map the process_comment function to the comments in the DataFrame batch
        replies = list(executor.map(process_comment, df_batch['Participant Comment']))
        # Add the replies to the DataFrame batch
        df_batch['Natural_Theme'], df_batch['Natural_Subtheme'] = zip(*replies)
        df_list.append(df_batch) # Add the processed DataFrame batch to the list

        print(f"Processed {batch_size} lines of survey") # Print a message after each batch is processed
        time.sleep(10) # Wait for 10 seconds before processing the next batch

# Concatenate all the processed DataFrame batches back into a single DataFrame
df = pd.concat(df_list)


Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey
Processed 10 lines of survey


In [100]:
        from IPython.display import display, HTML

        # Convert the DataFrame to an HTML table with borders
        html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

        # Display the HTML table
        display(HTML(html_table))

,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined,gpt_Mission,Natural_Theme,Natural_Subtheme
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The Waiting Times In The Emergency Department Are Too Long. This Often Leads To Patient Discomfort And Dissatisfaction.,Yes,Solution,Yes,Problem,None,[[Theme 1]],Quality & Safety,None,None
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A Triage System Could Help Prioritize Cases In The Emergency Department. It Would Ensure That Patients With The Most Urgent Needs Are Attended To First.,No,Neither,No,Solution,None,None,Quality & Safety,None,None
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The Hospital Is Understaffed During Night Shifts. This Can Lead To Delays In Patient Care And Increased Workload For Existing Staff.,No,NaN,Yes,Problem,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]",Quality & Safety,None,None
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring More Night Shift Nurses Could Alleviate Staffing Issues. It Would Ensure Adequate Care For All Patients And Reduce Burnout Among Existing Staff.,Yes,NaN,Yes,Neither,[Research & Discovery],[[Theme 1]],"Patient Care, Quality & Safety, Leadership",Human Resources,"[Hiring, Training, Benefits, Employee Relations]"
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The Hospital Food Lacks Variety And Taste. Patients Often Complain About The Lack Of Healthy And Appetizing Options.,No,NaN,No,Problem,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]",Patient Care,None,None
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The Hospital’S Parking Lot Is Often Full, Making It Difficult For Visitors. This Can Cause Stress For Visitors And May Deter Them From Visiting Their Loved Ones.",Yes,NaN,Yes,Problem,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]",Quality & Safety,None,None
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A Menu Revamp Could Improve Patient Satisfaction. Offering A Variety Of Tasty And Nutritious Meals Could Enhance The Overall Patient Experience.,Yes,NaN,No,Neither,[Teaching],[[Theme 6]],Patient Care,Patient Experience,"[Satisfaction, Comfort, Privacy, Communication with Staff, Wait Times]"
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A Multi-Level Parking Garage Could Solve The Parking Issue. It Would Provide Ample Parking Space For Visitors And Staff.,Yes,NaN,No,Neither,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]",Other,None,None
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The Hospital Rooms Are Not Cleaned Regularly. This Can Lead To Unsanitary Conditions And Potential Health Risks For Patients.,Yes,NaN,No,Problem,None,None,Quality & Safety,None,None
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing The Frequency Of Cleaning Could Improve Cleanliness. Regular Cleaning And Sanitation Are Crucial For Maintaining A Healthy Hospital Environment.,Yes,NaN,No,Neither,None,None,Quality & Safety,Buildings and Grounds,"[Maintenance, Cleaning, Security, Accessibility]"


I gated the last to 10 line per call. Very slow but need to batch at some level. 

In [101]:
# Capitalize the first letter of each word in every field of the DataFrame
df = df.applymap(lambda s: s.title() if isinstance(s, str) else s)

from IPython.display import display, HTML

# Convert the DataFrame to an HTML table with borders
html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

# Display the HTML table
display(HTML(html_table))



,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined,gpt_Mission,Natural_Theme,Natural_Subtheme
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The Waiting Times In The Emergency Department Are Too Long. This Often Leads To Patient Discomfort And Dissatisfaction.,Yes,Solution,Yes,Problem,None,[[Theme 1]],Quality & Safety,None,None
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A Triage System Could Help Prioritize Cases In The Emergency Department. It Would Ensure That Patients With The Most Urgent Needs Are Attended To First.,No,Neither,No,Solution,None,None,Quality & Safety,None,None
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The Hospital Is Understaffed During Night Shifts. This Can Lead To Delays In Patient Care And Increased Workload For Existing Staff.,No,NaN,Yes,Problem,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]",Quality & Safety,None,None
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring More Night Shift Nurses Could Alleviate Staffing Issues. It Would Ensure Adequate Care For All Patients And Reduce Burnout Among Existing Staff.,Yes,NaN,Yes,Neither,[Research & Discovery],[[Theme 1]],"Patient Care, Quality & Safety, Leadership",Human Resources,"[Hiring, Training, Benefits, Employee Relations]"
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The Hospital Food Lacks Variety And Taste. Patients Often Complain About The Lack Of Healthy And Appetizing Options.,No,NaN,No,Problem,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]",Patient Care,None,None
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The Hospital’S Parking Lot Is Often Full, Making It Difficult For Visitors. This Can Cause Stress For Visitors And May Deter Them From Visiting Their Loved Ones.",Yes,NaN,Yes,Problem,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]",Quality & Safety,None,None
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A Menu Revamp Could Improve Patient Satisfaction. Offering A Variety Of Tasty And Nutritious Meals Could Enhance The Overall Patient Experience.,Yes,NaN,No,Neither,[Teaching],[[Theme 6]],Patient Care,Patient Experience,"[Satisfaction, Comfort, Privacy, Communication with Staff, Wait Times]"
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A Multi-Level Parking Garage Could Solve The Parking Issue. It Would Provide Ample Parking Space For Visitors And Staff.,Yes,NaN,No,Neither,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]",Other,None,None
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The Hospital Rooms Are Not Cleaned Regularly. This Can Lead To Unsanitary Conditions And Potential Health Risks For Patients.,Yes,NaN,No,Problem,None,None,Quality & Safety,None,None
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing The Frequency Of Cleaning Could Improve Cleanliness. Regular Cleaning And Sanitation Are Crucial For Maintaining A Healthy Hospital Environment.,Yes,NaN,No,Neither,None,None,Quality & Safety,Buildings And Grounds,"[Maintenance, Cleaning, Security, Accessibility]"


In [102]:
import os
import openai
import numpy as np
import pandas as pd
import time
import concurrent.futures

openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = AZURE_OPENAI_ENDPOINT4 # Your Azure OpenAI resource's endpoint value.
openai.api_key = AZURE_OPENAI_API_KEY4

# Define a function to process a single comment
def process_comment(row):
    # Combine the participant comment, natural theme, and natural subtheme
    combined_comment = f"{row['Participant Comment']} {row['Natural_Theme']} {row['Natural_Subtheme']}"

    # Define a list of digital technologies
    digital_technologies = [
        "Electronic Health Records", "mHealth", "Telemedicine", "Telehealth", 
        "Artificial Intelligence", "AI", "Machine Learning", "Predictive Analytics", 
        "Virtual Reality", "VR", "Wearable Technology", "Real-Time Location Services", 
        "RTLS", "Robotics", "3D Printing", "Internet of Things", "IoT", 
        "Cloud Computing", "Cybersecurity", "Blockchain", "Big Data", 
        "Interoperability Solutions", "Other"
    ]
    
    # Identify the central digital technology in the combined comment
    for tech in digital_technologies:
        if tech.lower() in combined_comment.lower():
            return tech
    
    # If no digital technology is found in the comment, return 'Other'
    return 'Other'

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Break the DataFrame into smaller batches
    batch_size = 15 # Adjust this value based on your needs
    df_batches = np.array_split(df, len(df) // batch_size)
    
    df_list = [] # List to store the processed DataFrame batches
    
    for i, df_batch in enumerate(df_batches):
        # Use the executor to map the process_comment function to the rows in the DataFrame batch
        replies = list(executor.map(process_comment, df_batch.to_dict('records')))
        # Add the replies to the DataFrame batch
        df_batch['gpt_digital'] = replies
        df_list.append(df_batch) # Add the processed DataFrame batch to the list

        print(f"Processed {batch_size} lines of survey") # Print a message after each batch is processed
        time.sleep(10) # Wait for 10 seconds before processing the next batch

# Concatenate all the processed DataFrame batches back into a single DataFrame
df = pd.concat(df_list)


Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey


In [103]:
# Capitalize the first letter of each word in every field of the DataFrame
df = df.applymap(lambda s: s.title() if isinstance(s, str) else s)

from IPython.display import display, HTML

# Convert the DataFrame to an HTML table with borders
html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

# Display the HTML table
display(HTML(html_table))

,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined,gpt_Mission,Natural_Theme,Natural_Subtheme,gpt_digital
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The Waiting Times In The Emergency Department Are Too Long. This Often Leads To Patient Discomfort And Dissatisfaction.,Yes,Solution,Yes,Problem,None,[[Theme 1]],Quality & Safety,None,None,Ai
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A Triage System Could Help Prioritize Cases In The Emergency Department. It Would Ensure That Patients With The Most Urgent Needs Are Attended To First.,No,Neither,No,Solution,None,None,Quality & Safety,None,None,Other
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The Hospital Is Understaffed During Night Shifts. This Can Lead To Delays In Patient Care And Increased Workload For Existing Staff.,No,NaN,Yes,Problem,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]",Quality & Safety,None,None,Other
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring More Night Shift Nurses Could Alleviate Staffing Issues. It Would Ensure Adequate Care For All Patients And Reduce Burnout Among Existing Staff.,Yes,NaN,Yes,Neither,[Research & Discovery],[[Theme 1]],"Patient Care, Quality & Safety, Leadership",Human Resources,"[Hiring, Training, Benefits, Employee Relations]",Ai
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The Hospital Food Lacks Variety And Taste. Patients Often Complain About The Lack Of Healthy And Appetizing Options.,No,NaN,No,Problem,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]",Patient Care,None,None,Ai
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The Hospital’S Parking Lot Is Often Full, Making It Difficult For Visitors. This Can Cause Stress For Visitors And May Deter Them From Visiting Their Loved Ones.",Yes,NaN,Yes,Problem,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]",Quality & Safety,None,None,Other
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A Menu Revamp Could Improve Patient Satisfaction. Offering A Variety Of Tasty And Nutritious Meals Could Enhance The Overall Patient Experience.,Yes,NaN,No,Neither,[Teaching],[[Theme 6]],Patient Care,Patient Experience,"[Satisfaction, Comfort, Privacy, Communication with Staff, Wait Times]",Ai
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A Multi-Level Parking Garage Could Solve The Parking Issue. It Would Provide Ample Parking Space For Visitors And Staff.,Yes,NaN,No,Neither,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]",Other,None,None,Other
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The Hospital Rooms Are Not Cleaned Regularly. This Can Lead To Unsanitary Conditions And Potential Health Risks For Patients.,Yes,NaN,No,Problem,None,None,Quality & Safety,None,None,Other
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing The Frequency Of Cleaning Could Improve Cleanliness. Regular Cleaning And Sanitation Are Crucial For Maintaining A Healthy Hospital Environment.,Yes,NaN,No,Neither,None,None,Quality & Safety,Buildings And Grounds,"[Maintenance, Cleaning, Security, Accessibility]",Ai


In [105]:
import os
import openai
import numpy as np
import pandas as pd
import time
import concurrent.futures

openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = AZURE_OPENAI_ENDPOINT4 # Your Azure OpenAI resource's endpoint value.
openai.api_key = AZURE_OPENAI_API_KEY4

# Define a function to process a single opt-in response
def process_opt_in_response(response):
    yes_responses = ["yes", "positive", "sure"]
    no_responses = ["none", "blank", "i don't know", "i'm good", "n/a", "no opinion", "no other ideas", "none"]

    # Convert the response to lowercase
    response = response.lower()

    # Check if the response is in the yes_responses or no_responses list
    if response in yes_responses:
        return 'Yes'
    elif response in no_responses:
        return 'No'
    else:
        # If the response is not clearly 'yes' or 'no', use GPT to interpret it
        messages = [
            {
                "role": "system",
                "content": "\n".join([
                    "Input: yes\nOutput: Yes",
                    "Input: positive\nOutput: Yes",
                    "Input: sure\nOutput: Yes",
                    "Input: none\nOutput: No",
                    "Input: blank\nOutput: No",
                    "Input: i don't know\nOutput: No",
                    "Input: i'm good\nOutput: No",
                    "Input: n/a\nOutput: No",
                    "Input: no opinion\nOutput: No",
                    "Input: no other ideas\nOutput: No",
                    "Input: none\nOutput: No",
                    f"Input: {response}\nOutput: "
                ])
            },
            {
                "role": "user",
                "content": response
            }
        ]
        # Make a chat completion Request for the API
        gpt_response = openai.ChatCompletion.create(
            engine="gpt-4-32k",
            messages=messages
        )
        # Process the response from API
        gpt_reply = gpt_response['choices'][0]['message']['content'].strip().lower()

        # Return 'Yes' or 'No' based on the GPT reply
        return 'Yes' if gpt_reply == 'yes' else 'No'

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Break the DataFrame into smaller batches
    batch_size = 15 # Adjust this value based on your needs
    df_batches = np.array_split(df, len(df) // batch_size)
    
    df_list = [] # List to store the processed DataFrame batches
    
    for i, df_batch in enumerate(df_batches):
        # Use the executor to map the process_opt_in_response function to the 'opt-in response?' column in the DataFrame batch
        replies = list(executor.map(process_opt_in_response, df_batch['opt-in response?']))
        # Add the replies to the DataFrame batch
        df_batch['Opt-IN'] = replies
        df_list.append(df_batch) # Add the processed DataFrame batch to the list

        print(f"Processed {batch_size} lines of survey") # Print a message after each batch is processed
        time.sleep(10) # Wait for 10 seconds before processing the next batch

# Concatenate all the processed DataFrame batches back into a single DataFrame
df = pd.concat(df_list)


Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey
Processed 15 lines of survey


In [118]:
# Capitalize the first letter of each word in every field of the DataFrame
df = df.applymap(lambda s: s.title() if isinstance(s, str) else s)

from IPython.display import display, HTML

# Convert the DataFrame to an HTML table with borders
html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

# Display the HTML table
display(HTML(html_table))

,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined,gpt_Mission,Natural_Theme,Natural_Subtheme,gpt_digital,Opt-IN
0,Site 1,Service 10,2023-01-31 14:01:00.000,2023-01,3,The Waiting Times In The Emergency Department Are Too Long. This Often Leads To Patient Discomfort And Dissatisfaction.,Yes,Solution,Yes,Problem,None,[[Theme 1]],Quality & Safety,None,None,Ai,Yes
1,Site 2,Service 5,2023-02-02 14:00:59.995,2023-02,3,A Triage System Could Help Prioritize Cases In The Emergency Department. It Would Ensure That Patients With The Most Urgent Needs Are Attended To First.,No,Neither,No,Solution,None,None,Quality & Safety,None,None,Other,No
2,Site 2,Service 8,2023-02-04 14:00:59.995,2023-02,1,The Hospital Is Understaffed During Night Shifts. This Can Lead To Delays In Patient Care And Increased Workload For Existing Staff.,No,NaN,Yes,Problem,"[Patient Care, Research & Discovery, Teaching, Community, Other]","[[Theme 1], [Theme 1], [Theme 1], [Theme 1], [Theme 2], [Theme 3], [Theme 4], [Theme 5], [Theme 6], [Theme 7], [Theme 8], [Theme 9], [Theme 10], Other]",Quality & Safety,None,None,Other,No
3,Site 4,Service 4,2023-02-06 14:00:59.995,2023-02,2,Hiring More Night Shift Nurses Could Alleviate Staffing Issues. It Would Ensure Adequate Care For All Patients And Reduce Burnout Among Existing Staff.,Yes,NaN,Yes,Neither,[Research & Discovery],[[Theme 1]],"Patient Care, Quality & Safety, Leadership",Human Resources,"[Hiring, Training, Benefits, Employee Relations]",Ai,Yes
4,Site 6,Service 6,2023-02-08 14:00:59.995,2023-02,1,The Hospital Food Lacks Variety And Taste. Patients Often Complain About The Lack Of Healthy And Appetizing Options.,No,NaN,No,Problem,[Community],"[[Theme 2], [Theme 8], [Theme 9], [Theme 10]]",Patient Care,None,None,Ai,No
5,Site 2,Service 8,2023-02-10 14:00:59.995,2023-02,1,"The Hospital’S Parking Lot Is Often Full, Making It Difficult For Visitors. This Can Cause Stress For Visitors And May Deter Them From Visiting Their Loved Ones.",Yes,NaN,Yes,Problem,"[Patient Care, Research & Discovery]","[[Theme 1], [Theme 4], [Theme 10], Other]",Quality & Safety,None,None,Other,Yes
6,Site 4,Service 10,2023-02-12 14:00:59.995,2023-02,2,A Menu Revamp Could Improve Patient Satisfaction. Offering A Variety Of Tasty And Nutritious Meals Could Enhance The Overall Patient Experience.,Yes,NaN,No,Neither,[Teaching],[[Theme 6]],Patient Care,Patient Experience,"[Satisfaction, Comfort, Privacy, Communication with Staff, Wait Times]",Ai,Yes
7,Site 6,Service 12,2023-02-14 14:00:59.995,2023-02,3,A Multi-Level Parking Garage Could Solve The Parking Issue. It Would Provide Ample Parking Space For Visitors And Staff.,Yes,NaN,No,Neither,[Teaching],"[[Theme 6], [Theme 7], [Theme 8]]",Other,None,None,Other,Yes
8,Site 7,Service 1,2022-04-20 10:00:00.000,2022-04,4,The Hospital Rooms Are Not Cleaned Regularly. This Can Lead To Unsanitary Conditions And Potential Health Risks For Patients.,Yes,NaN,No,Problem,None,None,Quality & Safety,None,None,Other,Yes
9,Site 8,Service 1,2022-04-21 09:59:59.995,2022-04,3,Increasing The Frequency Of Cleaning Could Improve Cleanliness. Regular Cleaning And Sanitation Are Crucial For Maintaining A Healthy Hospital Environment.,Yes,NaN,No,Neither,None,None,Quality & Safety,Buildings And Grounds,"[Maintenance, Cleaning, Security, Accessibility]",Ai,Yes


In [108]:
df_backup = df.copy()
print(df_backup)

        Site     Dept/Div    submission timestamp year-month  Q ID  \
0     Site 1  Service 10  2023-01-31 14:01:00.000    2023-01     3   
1     Site 2   Service 5  2023-02-02 14:00:59.995    2023-02     3   
2     Site 2    Service 8 2023-02-04 14:00:59.995    2023-02     1   
3     Site 4    Service 4 2023-02-06 14:00:59.995    2023-02     2   
4     Site 6    Service 6 2023-02-08 14:00:59.995    2023-02     1   
..       ...          ...                     ...        ...   ...   
183  Site 11    Service 4 2022-09-19 09:59:59.995    2022-09     3   
184  Site 12    Service 4 2022-09-20 09:59:59.995    2022-09     3   
185  Site 13    Service 4 2022-09-21 09:59:59.995    2022-09     3   
186  Site 14    Service 4 2022-09-22 09:59:59.995    2022-09     3   
187  Site 15    Service 4 2022-09-23 09:59:59.995    2022-09     3   

                                   Participant Comment opt-in response?  \
0    The Waiting Times In The Emergency Department ...              Yes   
1    A Tr

In [115]:
# do this if the changes are not what i want
#df = df_backup.copy()
# print(df)

        Site     Dept/Div    submission timestamp year-month  Q ID  \
0     Site 1  Service 10  2023-01-31 14:01:00.000    2023-01     3   
1     Site 2   Service 5  2023-02-02 14:00:59.995    2023-02     3   
2     Site 2    Service 8 2023-02-04 14:00:59.995    2023-02     1   
3     Site 4    Service 4 2023-02-06 14:00:59.995    2023-02     2   
4     Site 6    Service 6 2023-02-08 14:00:59.995    2023-02     1   
..       ...          ...                     ...        ...   ...   
183  Site 11    Service 4 2022-09-19 09:59:59.995    2022-09     3   
184  Site 12    Service 4 2022-09-20 09:59:59.995    2022-09     3   
185  Site 13    Service 4 2022-09-21 09:59:59.995    2022-09     3   
186  Site 14    Service 4 2022-09-22 09:59:59.995    2022-09     3   
187  Site 15    Service 4 2022-09-23 09:59:59.995    2022-09     3   

                                   Participant Comment opt-in response?  \
0    The Waiting Times In The Emergency Department ...              Yes   
1    A Tr

In [119]:
print(df.columns)

Index(['Site', 'Dept/Div', 'submission timestamp', 'year-month', 'Q ID',
       'Participant Comment', 'opt-in response?', 'problem / solution?',
       'Large-Scale Initiative || Digital Technology', 'gpt_problem',
       'mission_defined', 'theme_defined', 'gpt_Mission', 'Natural_Theme',
       'Natural_Subtheme', 'gpt_digital', 'Opt-IN'],
      dtype='object')


In [128]:
# Remove brackets and quotes from 'Natural_Subtheme' column
df['Natural_Subtheme'] = df['Natural_Subtheme'].str.strip("[]").str.replace("'", "").str.replace('"', '')
df['mission_defined'] = df['mission_defined'].str.strip("[]").str.replace("'", "").str.replace('"', '')
df['theme_defined'] = df['theme_defined'].str.strip("[]").str.replace("'", "").str.replace('"', '')

In [120]:
import pandas as pd
import ast

# Assuming 'df' is your DataFrame
# Convert the 'Participant Comment' column to string type
df['Participant Comment'] = df['Participant Comment'].astype(str)

# Now you can apply string methods to it
df['Participant Comment'] = df['Participant Comment'].str.replace(r'[^\w\s]', '', regex=True)

# Convert lists to strings, remove brackets, and convert back to lists in 'mission_defined' and 'theme_defined' columns
df.loc[:, 'mission_defined'] = df.loc[:, 'mission_defined'].apply(lambda x: ast.literal_eval(str(x).replace('[', '').replace(']', '')) if isinstance(x, list) else x)
df.loc[:, 'theme_defined'] = df.loc[:, 'theme_defined'].apply(lambda x: ast.literal_eval(str(x).replace('[', '').replace(']', '')) if isinstance(x, list) else x)

# Convert 'Natural_Subtheme' column to string type only for non-null values
df.loc[df['Natural_Subtheme'].notna(), 'Natural_Subtheme'] = df.loc[df['Natural_Subtheme'].notna(), 'Natural_Subtheme'].astype(str)

# Now you can apply string methods to it
df.loc[:, 'Natural_Subtheme'] = df.loc[:, 'Natural_Subtheme'].str.replace(r'\\', '', regex=True)
df.loc[:, 'Natural_Subtheme'] = df.loc[:, 'Natural_Subtheme'].str.replace('\n', ' ')

# Convert the 'submission timestamp' column to datetime
df['submission timestamp'] = pd.to_datetime(df['submission timestamp'])

# Remove the time of day
df['submission timestamp'] = df['submission timestamp'].dt.date

# Replace any non-ASCII characters in 'Participant Comment' column
df['Participant Comment'] = df['Participant Comment'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii') if isinstance(x, str) else x)

# Check if 'Qid' column exists in the DataFrame and convert it to integer type
if 'Qid' in df.columns:
    df['Qid'] = df['Qid'].astype(int)






In [129]:
# Capitalize the first letter of each word in every field of the DataFrame
df = df.applymap(lambda s: s.title() if isinstance(s, str) else s)

from IPython.display import display, HTML

# Convert the DataFrame to an HTML table with borders
html_table = df.to_html().replace('<table>', '<table style="border:2px solid black;">')

# Display the HTML table
display(HTML(html_table))

,Site,Dept/Div,submission timestamp,year-month,Q ID,Participant Comment,opt-in response?,problem / solution?,Large-Scale Initiative || Digital Technology,gpt_problem,mission_defined,theme_defined,gpt_Mission,Natural_Theme,Natural_Subtheme,gpt_digital,Opt-IN
0,Site 1,Service 10,2023-01-31,2023-01,3,The Waiting Times In The Emergency Department Are Too Long This Often Leads To Patient Discomfort And Dissatisfaction,Yes,Solution,Yes,Problem,NaN,Theme 1,Quality & Safety,None,None,Ai,Yes
1,Site 2,Service 5,2023-02-02,2023-02,3,A Triage System Could Help Prioritize Cases In The Emergency Department It Would Ensure That Patients With The Most Urgent Needs Are Attended To First,No,Neither,No,Solution,NaN,NaN,Quality & Safety,None,None,Other,No
2,Site 2,Service 8,2023-02-04,2023-02,1,The Hospital Is Understaffed During Night Shifts This Can Lead To Delays In Patient Care And Increased Workload For Existing Staff,No,NaN,Yes,Problem,NaN,NaN,Quality & Safety,None,None,Other,No
3,Site 4,Service 4,2023-02-06,2023-02,2,Hiring More Night Shift Nurses Could Alleviate Staffing Issues It Would Ensure Adequate Care For All Patients And Reduce Burnout Among Existing Staff,Yes,NaN,Yes,Neither,Research & Discovery,Theme 1,"Patient Care, Quality & Safety, Leadership",Human Resources,"Hiring, Training, Benefits, Employee Relations",Ai,Yes
4,Site 6,Service 6,2023-02-08,2023-02,1,The Hospital Food Lacks Variety And Taste Patients Often Complain About The Lack Of Healthy And Appetizing Options,No,NaN,No,Problem,Community,NaN,Patient Care,None,None,Ai,No
5,Site 2,Service 8,2023-02-10,2023-02,1,The Hospitals Parking Lot Is Often Full Making It Difficult For Visitors This Can Cause Stress For Visitors And May Deter Them From Visiting Their Loved Ones,Yes,NaN,Yes,Problem,NaN,NaN,Quality & Safety,None,None,Other,Yes
6,Site 4,Service 10,2023-02-12,2023-02,2,A Menu Revamp Could Improve Patient Satisfaction Offering A Variety Of Tasty And Nutritious Meals Could Enhance The Overall Patient Experience,Yes,NaN,No,Neither,Teaching,Theme 6,Patient Care,Patient Experience,"Satisfaction, Comfort, Privacy, Communication With Staff, Wait Times",Ai,Yes
7,Site 6,Service 12,2023-02-14,2023-02,3,A Multilevel Parking Garage Could Solve The Parking Issue It Would Provide Ample Parking Space For Visitors And Staff,Yes,NaN,No,Neither,Teaching,NaN,Other,None,None,Other,Yes
8,Site 7,Service 1,2022-04-20,2022-04,4,The Hospital Rooms Are Not Cleaned Regularly This Can Lead To Unsanitary Conditions And Potential Health Risks For Patients,Yes,NaN,No,Problem,NaN,NaN,Quality & Safety,None,None,Other,Yes
9,Site 8,Service 1,2022-04-21,2022-04,3,Increasing The Frequency Of Cleaning Could Improve Cleanliness Regular Cleaning And Sanitation Are Crucial For Maintaining A Healthy Hospital Environment,Yes,NaN,No,Neither,NaN,NaN,Quality & Safety,Buildings And Grounds,"Maintenance, Cleaning, Security, Accessibility",Ai,Yes


In [130]:
        # Save the processed DataFrame to a CSV file
        df.to_csv(blob.name.replace('.xlsx', '.csv'), index=False)

        # Upload the processed file to the processed container
        with open(blob.name.replace('.xlsx', '.csv'), 'rb') as file:
            blob_service_client.get_blob_client(processed_container, blob.name.replace('.xlsx', '.csv')).upload_blob(file.read(), overwrite=True)

        # Move the raw processed file to the raw processed container
        blob_service_client.get_blob_client(raw_container_processed, blob.name).start_copy_from_url(source_container_client.get_blob_client(blob.name).url)
        source_container_client.delete_blob(blob.name)